In [1]:
# %pip install langchain Wikipedia-API

#### Imports

In [2]:
import time
from IPython.display import display, Markdown
from wikipediaapi import Wikipedia
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import Ollama

#### Data collection

In [3]:
wikipedia = Wikipedia('MyProjectName (merlin@example.com)', 'en')

def get_wikipedia_page(page_name):
	page = wikipedia.page(page_name)
	
	if page.exists() and len(page.text) > 0:
		return page.text
	else:
		return None

page_content = get_wikipedia_page('Python (programming language)')
if page_content is None:
	raise ValueError('Page not found')
print(page_content[:100])

Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code 


#### Creating the model

Before you run this, please install [Ollama](https://ollama.com/download) and run
`ollama pull` and `ollama pull llama2`

In [4]:
llm = Ollama(model="llama2")
llm

Ollama()

In [5]:
CHARACTER_LIMIT = 2000
# prompt can also be saved to a file and used as a template
prompt = """
content from wikipedia page is mentioned below. Answer the user's question using the content.
Keep the answers very short and to the point.
If user asks something not related to the content, avoid answering even partially
and apologize the user for not answering.
Avoid using technical jargon.
Avoid using words like "Hi", "According to", "In my opinion", "I think", "I believe", etc unless mandatory.
Keep the answers very short and to the point.
"""  # emphasizing on a short answer for a faster response and saving CPU time

prompt = prompt.strip().replace("\n", " ")

promptTemplate = ChatPromptTemplate.from_messages([
	("system", prompt),
	("system", page_content[:CHARACTER_LIMIT]),
	("user", "{question}"),
])
chain = promptTemplate | llm | StrOutputParser()

def answer_question(question):
	return chain.invoke({"question": question})

CHARACTERS_PER_LINE = 80

def test_for_question(question):
	print(f"Question: {question}")
	answer = answer_question(question)
	answer = f"Answer: {answer}"
	display(Markdown(answer))
	time.sleep(2)  # CPU cooldown break

print("Functions ready")

Functions ready


#### Testing with some queries

In [6]:
test_for_question("what is python?")

Question: what is python?


Answer: Python is a high-level, general-purpose programming language that emphasizes code readability with significant indentation. It's dynamically typed and garbage-collected, supporting multiple paradigms like procedural, object-oriented, and functional programming.

In [7]:
test_for_question("why python? why not javascript?")

Question: why python? why not javascript?


Answer: Python is a popular programming language due to its ease of use, readability, and extensive standard library. It's often preferred over JavaScript for several reasons:

1. Syntax: Python has a simpler syntax than JavaScript, making it easier to learn and understand.
2. Dynamically typed vs statically typed: Python is dynamically typed, which means you don't need to declare variables before using them. JavaScript, on the other hand, is statically typed, requiring explicit type declarations.
3. Memory management: Python uses automatic memory management through garbage collection, reducing the risk of memory leaks and making it easier to write programs that handle large amounts of data. JavaScript also has automatic memory management through its garbage collector, but it's not as efficient as Python's.
4. Cross-platform compatibility: Python can run on multiple operating systems without modification, while JavaScript is primarily associated with the web and can be more challenging to use on other platforms.
5. Libraries and frameworks: Python has a vast collection of libraries and frameworks for various tasks, such as machine learning, data analysis, and web development. JavaScript also has a rich ecosystem of libraries and frameworks, but Python's are often considered more mature and robust.
6. Learning curve: Many developers find Python easier to learn and use than JavaScript, especially for beginners.
7. Job prospects: Both languages have strong job prospects, but Python is often preferred in the industry due to its versatility and wide range of applications.

Ultimately, the choice between Python and JavaScript depends on your specific needs, goals, and preferences as a developer.

In [8]:
# Unrelated question
test_for_question("what is garbage collector in java?")

Question: what is garbage collector in java?


Answer: A garbage collector in Java is a component that automatically reclaims memory occupied by objects that are no longer needed or that have reached the end of their lifecycle. It frees up memory for use by other objects, improving the overall performance and efficiency of the program.

In [9]:
# Asking in Spanish - who invented python
test_for_question("quien inventó python")

Question: quien inventó python


Answer: Guido van Rossum es el creador de Python.

In [10]:
# Asking in Hindi - who invented python
test_for_question("पाइथॉन का आविष्कार किसने किया")

Question: पाइथॉन का आविष्कार किसने किया


Answer: Guido van Rossum conceived Python in the late 1980s as a successor to the ABC programming language.